# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pandas.io.json import json_normalize
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
# create a Spark session
spark = SparkSession\
                    .builder\
                    .master('local')\
                    .appName("Project")\
                    .getOrCreate();

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# Loadin the json file into a dataframe
df = spark.read.json('mini_sparkify_event_data.json')

In [4]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
print((df.count(), len(df.columns)))

(286500, 18)


In [7]:
df.toPandas().isnull().sum()

artist           58392
auth                 0
firstName         8346
gender            8346
itemInSession        0
lastName          8346
length           58392
level                0
location          8346
method               0
page                 0
registration      8346
sessionId            0
song             58392
status               0
ts                   0
userAgent         8346
userId               0
dtype: int64

In [8]:
df = df.dropna( subset = ['userId', 'sessionId'], how = 'any')

In [9]:
df.toPandas().isnull().sum()

artist           58392
auth                 0
firstName         8346
gender            8346
itemInSession        0
lastName          8346
length           58392
level                0
location          8346
method               0
page                 0
registration      8346
sessionId            0
song             58392
status               0
ts                   0
userAgent         8346
userId               0
dtype: int64

In [10]:
df = df.filter(df['userId'] != '')

In [11]:
df.toPandas().isnull().sum()

artist           50046
auth                 0
firstName            0
gender               0
itemInSession        0
lastName             0
length           50046
level                0
location             0
method               0
page                 0
registration         0
sessionId            0
song             50046
status               0
ts                   0
userAgent            0
userId               0
dtype: int64

In [12]:
df.dtypes

[('artist', 'string'),
 ('auth', 'string'),
 ('firstName', 'string'),
 ('gender', 'string'),
 ('itemInSession', 'bigint'),
 ('lastName', 'string'),
 ('length', 'double'),
 ('level', 'string'),
 ('location', 'string'),
 ('method', 'string'),
 ('page', 'string'),
 ('registration', 'bigint'),
 ('sessionId', 'bigint'),
 ('song', 'string'),
 ('status', 'bigint'),
 ('ts', 'bigint'),
 ('userAgent', 'string'),
 ('userId', 'string')]

In [13]:
df = df.withColumn("userId", df["userId"].cast(IntegerType()))

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [14]:
df.select('page','UserId').groupby('page').agg({'page':'count'}).select('page','count(page)').show()

+--------------------+-----------+
|                page|count(page)|
+--------------------+-----------+
|              Cancel|         52|
|    Submit Downgrade|         63|
|         Thumbs Down|       2546|
|                Home|      10082|
|           Downgrade|       2055|
|         Roll Advert|       3933|
|              Logout|       3226|
|       Save Settings|        310|
|Cancellation Conf...|         52|
|               About|        495|
|            Settings|       1514|
|     Add to Playlist|       6526|
|          Add Friend|       4277|
|            NextSong|     228108|
|           Thumbs Up|      12551|
|                Help|       1454|
|             Upgrade|        499|
|               Error|        252|
|      Submit Upgrade|        159|
+--------------------+-----------+



We will be using the "Cancellation Confirmation" page to define the churn

In [15]:
churn = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("churn", churn("page"))
windowvalue = Window.partitionBy("userId")
df = df.withColumn("churn", Fsum("churn").over(windowvalue))

In [16]:
df.take(5)

[Row(artist=None, auth='Logged In', firstName='Evan', gender='M', itemInSession=0, lastName='Spencer', length=None, level='free', location='Miami-Fort Lauderdale-West Palm Beach, FL', method='GET', page='Home', registration=1537531076000, sessionId=147, song=None, status=200, ts=1538805488000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId=148, churn=0),
 Row(artist='Blues Traveler', auth='Logged In', firstName='Evan', gender='M', itemInSession=1, lastName='Spencer', length=290.24608, level='free', location='Miami-Fort Lauderdale-West Palm Beach, FL', method='PUT', page='NextSong', registration=1537531076000, sessionId=147, song='Hook', status=200, ts=1538805521000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId=148, churn=0),
 Row(artist='Sheena Easton', auth='Logged In', firstName='Evan', gender='M', itemInSession=2, lastName='Spencer', length=239.62077, level='free', location='Miami-Fort Laud

In [17]:
df.createOrReplaceTempView('temp')

In [18]:
spark.sql(""" select count(DISTINCT userId) from temp""").show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   225|
+----------------------+



In [19]:
spark.sql(""" select churn,count(DISTINCT userId) from temp group by churn""").show()

+-----+----------------------+
|churn|count(DISTINCT userId)|
+-----+----------------------+
|    0|                   173|
|    1|                    52|
+-----+----------------------+



This table show that we have 52 users who have churned and 173 users who haven't

In [20]:
#This code will look at the differance between users who churned vs users who have not.

# label                   - represent the churn where 1 = churn, and 0 = not churned
# song_count              - represent the number of songs a user have listned to
# error                   - represent the number times the user have gotten the error page
# friends                 - represent the number of friends a user have added
# playlist_count          - represent the number of times the user have visited the playlist page
# thumbs_up               - represent the number of times a user have clicked the thumbs up button
# thumbs_down             - represent the number of times a user have clicked the thumbs down button
# males                   - represent the number of male users
# females                 - represent the number of female users
# free_user               - represent the number of users with free accounts
# paid_user               - represent the number of users with paid accounts
# count_session_dist      - represent the number of session a user have
# duration                - represent the days since the users have registared
# usage_time              - represent the time the user have spent in Sparkify




diff = spark.sql(" WITH prep as( \
SELECT userId, \
max(churn)                                                          as label, \
count(case when page = 'NextSong' then userId else null end)        as song_count, \
count(case when page = 'Error' then userId else null end)           as error, \
count(case when page = 'Add Friend' then userId else null end)      as friends, \
count(case when page = 'Add to Playlist' then userId else null end) as playlist_count, \
count(case when page = 'Thumbs Up' then userId else null end)       as thumbs_up, \
count(case when page = 'Thumbs Down' then userId else null end)     as thumbs_down, \
count(case when gender = 'M' then userId else null end)             as males, \
count(case when gender = 'F' then userId else null end)             as females, \
count(case when level = 'free' then userId else null end)           as free_user, \
count(case when level = 'paid' then userId else null end)           as paid_user, \
count(distinct sessionId)                                           as count_session_dist, \
(max(ts) - min(registration))/86400                                 as duration, \
sum(length)                                                         as usage_time \
FROM temp \
GROUP BY userId) \
SELECT label, \
count(label)             as churn_count, \
avg(song_count)          as song_count, \
avg(error)               as error, \
avg(friends)             as friends, \
avg(playlist_count)      as playlist_count, \
avg(thumbs_up)           as thumbs_up, \
avg(thumbs_down)         as thumbs_down, \
avg(males)               as males, \
avg(females)             as females, \
avg(free_user)           as free_user, \
avg(paid_user)           as paid_user, \
avg(count_session_dist)  as count_session_dist, \
avg(duration)            as duration, \
avg(usage_time)          as usage_time \
FROM prep \
GROUP BY label")

diff.toPandas()

,label,churn_count,song_count,error,friends,playlist_count,thumbs_up,thumbs_down,males,females,free_user,paid_user,count_session_dist,duration,usage_time
0,0,173,1108.173410,1.271676,21.046243,31.722543,61.803468,11.849711,565.780347,782.716763,250.479769,1098.017341,15.254335,86620.619380,276166.937468
1,1,52,699.884615,0.615385,12.230769,19.961538,35.750000,9.538462,494.153846,368.615385,238.230769,624.538462,10.326923,57305.992922,174014.268551


# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [21]:
# Creating the data that will be used as input for the model
features = spark.sql("SELECT userId, \
max(churn)                                                          as label, \
count(case when page = 'NextSong' then userId else null end)        as song_count, \
count(case when page = 'Error' then userId else null end)           as error, \
count(case when page = 'Add Friend' then userId else null end)      as friends, \
count(case when page = 'Add to Playlist' then userId else null end) as playlist_count, \
count(case when page = 'Thumbs Up' then userId else null end)       as thumbs_up, \
count(case when page = 'Thumbs Down' then userId else null end)     as thumbs_down, \
count(case when gender = 'M' then userId else null end)             as males, \
count(case when gender = 'F' then userId else null end)             as females, \
count(case when level = 'free' then userId else null end)           as free_user, \
count(case when level = 'paid' then userId else null end)           as paid_user, \
count(distinct sessionId)                                           as count_session_dist, \
(max(ts) - min(registration))/86400                                 as duration, \
sum(length)                                                         as usage_time \
FROM temp \
GROUP BY userId");

features.createOrReplaceTempView('features');
features=features.na.drop()

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [22]:
# Split the data into train and test
train, test = features.randomSplit([0.7, 0.3], seed=123)

In [23]:
assemble = VectorAssembler(inputCols=["userId","song_count","error","friends","playlist_count", \
                                       "thumbs_up","thumbs_down","males","females","free_user","paid_user", "count_session_dist",\
                                       "duration",\
                                       "usage_time"],\
                            outputCol="inputFeatures")



# Normalize Data
scale= Normalizer(inputCol="inputFeatures", outputCol="features")

In [24]:
# Building the model
pipeline=Pipeline(stages=[assemble, scale, RandomForestClassifier()])
paramgrid =ParamGridBuilder()\
.build()

evaluator=MulticlassClassificationEvaluator(metricName="f1")

crossval= CrossValidator(estimator=pipeline,  
                         estimatorParamMaps=paramgrid,
                         evaluator=evaluator, 
                         numFolds=10
                        )

cv_model=crossval.fit(train) 
evaluator.evaluate(cv_model.transform(test))

0.6894247833803111

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.